In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import files
from skimage import io

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
from shutil import copyfile
copyfile('/gdrive/MyDrive/SoML-50.zip', 'data.zip')

'data.zip'

In [ ]:
!unzip data.zip

In [ ]:
df = pd.read_csv("/content/SoML-50/annotations.csv")

In [ ]:
data = df.values.tolist()

In [ ]:
prefix = [i for i in data if i[1] == 'prefix']
postfix = [i for i in data if i[1] == 'postfix']
infix = [i for i in data if i[1] == 'infix']

In [ ]:
len(prefix), len(postfix), len(infix)

(16666, 16666, 16668)

In [ ]:
from random import shuffle
shuffle(prefix)
shuffle(postfix)
shuffle(infix)

In [ ]:
prefix_train, prefix_test = prefix[:15000], prefix[15000:]
postfix_train, postfix_test = postfix[:15000], postfix[15000:]
infix_train, infix_test = infix[:15000], infix[15000:]

In [ ]:
!mkdir final
! mkdir final/train_dir
! mkdir final/test_dir
! mkdir final/test_dir/prefix
! mkdir final/test_dir/postfix
! mkdir final/test_dir/infix
! mkdir final/train_dir/prefix
! mkdir final/train_dir/postfix
! mkdir final/train_dir/infix

In [ ]:
prefix_train[0]

['5524.jpg', 'prefix', 7]

In [ ]:
def copy_all(array, folder):
  for i in array:
    copyfile("/content/SoML-50/data/" + i[0], folder + i[0])

In [ ]:
copy_all(prefix_train, "/content/final/train_dir/prefix/")
copy_all(postfix_train, "/content/final/train_dir/postfix/")
copy_all(infix_train, "/content/final/train_dir/infix/")

copy_all(prefix_test, "/content/final/test_dir/prefix/")
copy_all(postfix_test, "/content/final/test_dir/postfix/")
copy_all(infix_test, "/content/final/test_dir/infix/")

# **We've data in the required format.**
# **Time to Train**

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    "/content/final/train_dir", 
    target_size=(128, 384),
    color_mode="grayscale",
    batch_size=128,
    class_mode='categorical'
)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    "/content/final/test_dir", 
    target_size=(128, 384),
    color_mode="grayscale",
    batch_size=128,
    class_mode='categorical'
)

Found 45000 images belonging to 3 classes.
Found 5000 images belonging to 3 classes.


In [ ]:
                            #  tf.keras.layers.Conv2D(256, (3, 3), activation='relu', input_shape=(128, 384, 1)),
                            #  tf.keras.layers.MaxPooling2D(2, 2),

model = tf.keras.Sequential([
                             tf.keras.layers.Conv2D(128, (3, 3), activation='relu', input_shape=(128, 384, 1)),
                             tf.keras.layers.MaxPooling2D(2, 2),
                             tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
                             tf.keras.layers.MaxPooling2D(2, 2),
                             tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
                             tf.keras.layers.MaxPooling2D(2, 2),
                             tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
                             tf.keras.layers.MaxPooling2D(2, 2),
                             tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(128, activation='relu'),
                             tf.keras.layers.Dense(64, activation='relu'),
                             tf.keras.layers.Dense(3, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 126, 382, 128)     1280      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 63, 191, 128)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 61, 189, 64)       73792     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 30, 94, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 28, 92, 32)        18464     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 14, 46, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 12, 44, 32)       

In [ ]:
history = model.fit(
    train_generator, 
    steps_per_epoch=352,
    epochs=10,
    validation_data=test_generator,
    validation_steps=8,
    verbose=1
)

Epoch 1/10
352/352 [==============================] - 149s 422ms/step - loss: 0.1161 - accuracy: 0.9508 - val_loss: 0.0056 - val_accuracy: 0.9990
Epoch 2/10
352/352 [==============================] - 148s 421ms/step - loss: 0.0036 - accuracy: 0.9987 - val_loss: 0.0070 - val_accuracy: 0.9971
Epoch 3/10
352/352 [==============================] - 148s 421ms/step - loss: 0.0023 - accuracy: 0.9992 - val_loss: 0.0028 - val_accuracy: 0.9980
Epoch 4/10
352/352 [==============================] - 148s 420ms/step - loss: 0.0029 - accuracy: 0.9991 - val_loss: 0.0068 - val_accuracy: 0.9980
Epoch 5/10
352/352 [==============================] - 148s 420ms/step - loss: 2.8070e-04 - accuracy: 0.9999 - val_loss: 0.0014 - val_accuracy: 0.9990
Epoch 6/10
352/352 [==============================] - 148s 420ms/step - loss: 5.4651e-04 - accuracy: 0.9998 - val_loss: 8.0507e-06 - val_accuracy: 1.0000
Epoch 7/10
352/352 [==============================] - 148s 420ms/step - loss: 4.9050e-06 - accuracy: 1.0000 - va

In [ ]:
train_generator.class_indices

{'infix': 0, 'postfix': 1, 'prefix': 2}

In [ ]:
def check():
  uploaded = files.upload()

  name = list(uploaded.keys())[0]
  print(name, "uploaded")
  img = keras.preprocessing.image.load_img(
    name, target_size=(128, 384), color_mode='grayscale'
  )
  img_array = keras.preprocessing.image.img_to_array(img)
  img_array = tf.expand_dims(img_array, 0)
  predictions = model.predict(img_array)
  score = tf.nn.softmax(predictions[0])
  a = np.argmax(score)
  if (a == 0):
    print("INFIX")
  elif a == 1:
    print("POSTFIX")
  elif a == 2:
    print("PREFIX")

In [ ]:
model.save("model/")

INFO:tensorflow:Assets written to: model/assets


In [ ]:
mod = keras.models.load_model("model/")

In [ ]:
def check_saved():
  uploaded = files.upload()

  name = list(uploaded.keys())[0]
  print(name, "uploaded")
  img = keras.preprocessing.image.load_img(
    name, target_size=(128, 384), color_mode='grayscale'
  )
  img_array = keras.preprocessing.image.img_to_array(img)
  img_array = tf.expand_dims(img_array, 0)
  predictions = mod.predict(img_array)
  score = tf.nn.softmax(predictions[0])
  a = np.argmax(score)
  if (a == 0):
    print("INFIX")
  elif a == 1:
    print("POSTFIX")
  elif a == 2:
    print("PREFIX")

In [ ]:
check_saved()

Saving 10017.jpg to 10017.jpg
10017.jpg uploaded
INFIX


# **Predicting a folder**

In [ ]:
from tensorflow import keras
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
import os
import argparse

# parser = argparse.ArgumentParser()
# parser.add_argument("folder", type=str)
# args = parser.parse_args()

# folder_path = args.folder # image folder

folder_path = "infix/"
model_path = 'model/' # model folder

img_width, img_height = 384, 128

model = load_model(model_path) # load the trained model

images = []
for img in os.listdir(folder_path):
    img = os.path.join(folder_path, img)
    img = image.load_img(img, target_size=(img_height, img_width),color_mode='grayscale')
    img = image.img_to_array(img)
    img /= 255.
    img = np.expand_dims(img, axis=0)
    images.append(img)

# stack up images list to pass for prediction
images = np.vstack(images)
classes = model.predict_classes(images, batch_size=10)
k = list(classes)
print(k)
print(k.count(0), k.count(1), k.count(2))

In [ ]:
! zip -r model.zip model

  adding: model/ (stored 0%)
  adding: model/keras_metadata.pb (deflated 93%)
  adding: model/variables/ (stored 0%)
  adding: model/variables/variables.index (deflated 69%)
  adding: model/variables/variables.data-00000-of-00001 (deflated 9%)
  adding: model/saved_model.pb (deflated 89%)
  adding: model/assets/ (stored 0%)


In [ ]:
python inference.py final/test_dir/prefix/

UsageError: %%python is a cell magic, but the cell body is empty.


In [ ]:
!unzip model.zip

Archive:  model.zip
   creating: model/
  inflating: model/keras_metadata.pb  
   creating: model/variables/
  inflating: model/variables/variables.index  
  inflating: model/variables/variables.data-00000-of-00001  
  inflating: model/saved_model.pb    
   creating: model/assets/
